## Imports

In [12]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)

In [13]:
import pandas as pd
import numpy as np
import catboost
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [14]:
bracket_training = pd.read_csv('bracket_training_with_names.csv')
bracket_training.head()

,Unnamed: 0,CustomerID,CustomerAreaCode,CustomerPostalCode,CustomerPostalCodeLatitude,CustomerPostalCodeLongitude,CustomerDMACode,CustomerDMADescription,NCAACustomerRecordCreated,BracketEntryId,BracketEntryCreatedDate,RegionWinner_East,RegionWinner_West,RegionWinner_South,RegionWinner_Midwest,SemifinalWinner_East_West,SemifinalWinner_South_Midwest,NationalChampion
0,0,47028,NaN,36093,32.5622,-86.0994,698.0,MONTGOMERY (SELMA),2021-12-25,1723503,2024-03-19 10:27:15 -0400,UConn,Arizona,James Madison,Tennessee,UConn,Tennessee,Tennessee
1,1,3511,616.0,49464,42.8256,-86.0104,563.0,GRAND RAPIDS - KALMZOO - B. CRK,2021-04-02,963479,2024-03-18 10:16:39 -0400,UConn,Baylor,Kentucky,Kansas,UConn,Kentucky,UConn
2,2,58445,703.0,22210,38.8808,-77.1129,511.0,"WASHINGTON, DC (HAGRSTWN)",2021-04-02,810038,2024-03-18 00:21:47 -0400,UConn,Baylor,Houston,Purdue,UConn,Purdue,Purdue
3,3,28833,NaN,78218,29.4969,-98.4032,641.0,SAN ANTONIO,2023-11-16,3384825,2024-03-21 10:28:56 -0400,Iowa St.,Saint Mary's,NC State,Purdue,Iowa St.,NC State,NC State
4,4,37899,NaN,14212,42.8946,-78.8245,514.0,BUFFALO,2022-03-16,2828017,2024-03-20 20:14:52 -0400,Auburn,North Carolina,Marquette,Creighton,North Carolina,Marquette,North Carolina


In [15]:
institutions = pd.read_csv('CCAC 2025 - Institutions.csv')
institutions.head()

,InstitutionID,InstitutionName,InstitutionNickname,InstitutionAbbreviation,InstitutionCity,InstitutionState,InstitutionPostalCode,InstitutionDMACode,InstitutionDMADescription,InstitutionLatitude,InstitutionLongitude,InstitutionConference,InstitutionEnrollment_Male,InstitutionEnrollment_Female,InstitutionEnrollment_Total,InstitutionNCAAMemberSinceDate,RegularSeasonWins,RegularSeasonLosses,RegularSeasonAverageAttendance,RegularSeasonAverageScore
0,626,San Diego St.,Aztecs,SDSU,San Diego,CA,92182,825,SAN DIEGO,32.77525,-117.07120,Mountain West Conference,11992,16456,28448,09/01/1947,24,10,9832.94,74.59
1,402,McNeese,Cowboys,MCNEES,Lake Charles,LA,70609,643,LAKE CHARLES,30.17819,-93.21722,Southland Conference,2245,3462,5707,09/01/1965,30,3,3085.58,80.42
2,648,South Carolina,Gamecocks,SO CAR,Columbia,SC,29208,546,"COLUMBIA, SC",33.99679,-81.02693,Southeastern Conference,11273,14774,26047,09/01/1936,26,7,11865.45,72.03
3,175,Dayton,Flyers,DAY,Dayton,OH,45469-1230,542,DAYTON,39.73955,-84.17611,Atlantic 10 Conference,4245,3872,8117,09/01/1951,24,8,9187.23,74.74
4,317,James Madison,Dukes,JMU,Harrisonburg,VA,22807,569,HARRISONBURG,38.43631,-78.87048,Sun Belt Conference,8342,12004,20346,09/01/1969,31,3,4471.35,84.35


In [16]:
bracket_test = pd.read_csv('bracket_test_with_names.csv')
bracket_test['SemifinalWinner_East_West'] = ""
bracket_test['SemifinalWinner_South_Midwest'] = ""
bracket_test.head()

,Unnamed: 0,CustomerID,CustomerAreaCode,CustomerPostalCode,CustomerPostalCodeLatitude,CustomerPostalCodeLongitude,CustomerDMACode,CustomerDMADescription,NCAACustomerRecordCreated,BracketEntryId,BracketEntryCreatedDate,RegionWinner_East,RegionWinner_West,RegionWinner_South,RegionWinner_Midwest,SemifinalWinner_East_West,SemifinalWinner_South_Midwest
0,0,73662,919.0,27539,35.7225,-78.8408,560.0,RALEIGH - DURHAM (FAYETVLLE),3/29/20,2074118,3/19/24 18:50,UConn,North Carolina,Houston,Purdue,,
1,1,6679,360.0,97206,45.4840,-122.5973,820.0,"PORTLAND, OR",4/2/24,2692634,3/20/24 16:56,UConn,North Carolina,Duke,Kansas,,
2,2,63024,270.0,42754,37.4603,-86.3249,529.0,LOUISVILLE,12/8/21,1252684,3/18/24 15:13,Iowa St.,Arizona,Kentucky,Creighton,,
3,3,60371,206.0,98178,47.4924,-122.2359,819.0,SEATTLE - TACOMA,3/22/23,1950205,3/19/24 15:21,UConn,North Carolina,Houston,Purdue,,
4,4,18415,717.0,19038,40.1096,-75.1550,504.0,PHILADELPHIA,2/20/24,2756293,3/20/24 18:40,UConn,North Carolina,Marquette,Creighton,,


In [17]:
bracket_training.shape

(130002, 18)

In [18]:
bracket_test.shape

(14445, 17)

## Semifinal 1 Predictions

### Master Training Dataframe

In [19]:
bracket_training_simple = bracket_training[['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_West', 'SemifinalWinner_East_West', 'BracketEntryId', 'NCAACustomerRecordCreated', 'BracketEntryCreatedDate']]
bracket_training_simple['CustomerDMACode'] = bracket_training_simple['CustomerDMACode'].astype(str).str[:-2]
bracket_training_simple['BracketEntryCreatedDate'] = bracket_training_simple['BracketEntryCreatedDate'].astype(str).str.split(' ').str[0]

bracket_training_simple['BracketEntryCreatedDate'] = pd.to_datetime(bracket_training_simple['BracketEntryCreatedDate'], infer_datetime_format=True, errors='coerce')
bracket_training_simple['NCAACustomerRecordCreated'] = pd.to_datetime(bracket_training_simple['NCAACustomerRecordCreated'], infer_datetime_format=True, errors='coerce')

# Calculate the number of months between the two dates
bracket_training_simple['MonthsBetween'] = ((bracket_training_simple['BracketEntryCreatedDate'].dt.year - bracket_training_simple['NCAACustomerRecordCreated'].dt.year) * 12 +
                                            (bracket_training_simple['BracketEntryCreatedDate'].dt.month - bracket_training_simple['NCAACustomerRecordCreated'].dt.month))

# Merge additional data for RegionWinner_East
bracket_training_simple = bracket_training_simple.merge(
    institutions[['InstitutionName', 'RegularSeasonWins', 'RegularSeasonLosses', 'RegularSeasonAverageAttendance', 'RegularSeasonAverageScore', 'InstitutionEnrollment_Male', 'InstitutionEnrollment_Female']],
    left_on='RegionWinner_East', right_on='InstitutionName', how='left'
).rename(columns={'RegularSeasonWins': 'RegionWinner_East_Wins', 'RegularSeasonLosses': 'RegionWinner_East_Losses',
                  'RegularSeasonAverageAttendance': 'RegionWinner_East_AvgAttendance', 'RegularSeasonAverageScore': 'RegionWinner_East_AvgScore',
                  'InstitutionEnrollment_Male': 'RegionWinner_East_Enrollment_Male', 'InstitutionEnrollment_Female': 'RegionWinner_East_Enrollment_Female'}).drop(columns='InstitutionName')

# Merge additional data for RegionWinner_West
bracket_training_simple = bracket_training_simple.merge(
    institutions[['InstitutionName', 'RegularSeasonWins', 'RegularSeasonLosses', 'RegularSeasonAverageAttendance', 'RegularSeasonAverageScore', 'InstitutionEnrollment_Male', 'InstitutionEnrollment_Female']],
    left_on='RegionWinner_West', right_on='InstitutionName', how='left'
).rename(columns={'RegularSeasonWins': 'RegionWinner_West_Wins', 'RegularSeasonLosses': 'RegionWinner_West_Losses',
                  'RegularSeasonAverageAttendance': 'RegionWinner_West_AvgAttendance', 'RegularSeasonAverageScore': 'RegionWinner_West_AvgScore',
                  'InstitutionEnrollment_Male': 'RegionWinner_West_Enrollment_Male', 'InstitutionEnrollment_Female': 'RegionWinner_West_Enrollment_Female'}).drop(columns='InstitutionName')

# Calculate win-loss ratio for RegionWinner_East
bracket_training_simple['RegionWinner_East_WinLossRatio'] = bracket_training_simple['RegionWinner_East_Wins'] / (bracket_training_simple['RegionWinner_East_Losses'] + 1e-9)

# Calculate win-loss ratio for RegionWinner_West
bracket_training_simple['RegionWinner_West_WinLossRatio'] = bracket_training_simple['RegionWinner_West_Wins'] / (bracket_training_simple['RegionWinner_West_Losses'] + 1e-9)

# Rearrange columns to group RegionWinner_East and RegionWinner_West columns together
bracket_training_simple = bracket_training_simple[['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_East_Wins', 'RegionWinner_East_Losses', 'RegionWinner_East_WinLossRatio',
                                                   'RegionWinner_East_AvgAttendance', 'RegionWinner_East_AvgScore', 'RegionWinner_East_Enrollment_Male', 'RegionWinner_East_Enrollment_Female',
                                                   'RegionWinner_West', 'RegionWinner_West_Wins', 'RegionWinner_West_Losses', 'RegionWinner_West_WinLossRatio',
                                                   'RegionWinner_West_AvgAttendance', 'RegionWinner_West_AvgScore', 'RegionWinner_West_Enrollment_Male', 'RegionWinner_West_Enrollment_Female',
                                                   'SemifinalWinner_East_West', 'BracketEntryId', 'NCAACustomerRecordCreated', 'BracketEntryCreatedDate', 'MonthsBetween']]

bracket_training_simple.set_index('BracketEntryId', inplace=True)
bracket_training_simple.head()

,CustomerDMACode,RegionWinner_East,RegionWinner_East_Wins,RegionWinner_East_Losses,RegionWinner_East_WinLossRatio,RegionWinner_East_AvgAttendance,RegionWinner_East_AvgScore,RegionWinner_East_Enrollment_Male,RegionWinner_East_Enrollment_Female,RegionWinner_West,...,RegionWinner_West_Losses,RegionWinner_West_WinLossRatio,RegionWinner_West_AvgAttendance,RegionWinner_West_AvgScore,RegionWinner_West_Enrollment_Male,RegionWinner_West_Enrollment_Female,SemifinalWinner_East_West,NCAACustomerRecordCreated,BracketEntryCreatedDate,MonthsBetween
BracketEntryId,,,,,,,,,,,,,,,,,,,,,
1723503,698,UConn,31,3,10.333333,14017.88,81.47,10645,11834,Arizona,...,8,3.125000,12118.18,87.94,13090,17292,UConn,2021-12-25,2024-03-19,27
963479,563,UConn,31,3,10.333333,14017.88,81.47,10645,11834,Baylor,...,10,2.300000,9517.79,80.55,6008,8877,UConn,2021-04-02,2024-03-18,35
810038,511,UConn,31,3,10.333333,14017.88,81.47,10645,11834,Baylor,...,10,2.300000,9517.79,80.55,6008,8877,UConn,2021-04-02,2024-03-18,35
3384825,641,Iowa St.,27,7,3.857143,12059.44,75.56,14070,11171,Saint Mary's,...,7,3.714286,3458.15,74.24,859,1090,Iowa St.,2023-11-16,2024-03-21,4
2828017,514,Auburn,27,7,3.857143,10182.91,83.32,12695,12684,North Carolina,...,7,3.857143,15767.47,81.47,8068,12174,North Carolina,2022-03-16,2024-03-20,24


In [20]:
bracket_training_simple.to_csv('bracket_training_master.csv')


### Master Test Dataframe

In [21]:
bracket_test_simple = bracket_test[['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_West', 'SemifinalWinner_East_West', 'BracketEntryId', 'NCAACustomerRecordCreated', 'BracketEntryCreatedDate']]
bracket_test_simple['CustomerDMACode'] = bracket_test_simple['CustomerDMACode'].astype(str).str[:-2]
bracket_test_simple['BracketEntryCreatedDate'] = bracket_test_simple['BracketEntryCreatedDate'].astype(str).str.split(' ').str[0]

bracket_test_simple['BracketEntryCreatedDate'] = pd.to_datetime(bracket_test_simple['BracketEntryCreatedDate'], infer_datetime_format=True, errors='coerce')
bracket_test_simple['NCAACustomerRecordCreated'] = pd.to_datetime(bracket_test_simple['NCAACustomerRecordCreated'], infer_datetime_format=True, errors='coerce')

# Calculate the number of months between the two dates
bracket_test_simple['MonthsBetween'] = ((bracket_test_simple['BracketEntryCreatedDate'].dt.year - bracket_test_simple['NCAACustomerRecordCreated'].dt.year) * 12 +
                                        (bracket_test_simple['BracketEntryCreatedDate'].dt.month - bracket_test_simple['NCAACustomerRecordCreated'].dt.month))

# Merge additional data for RegionWinner_East
bracket_test_simple = bracket_test_simple.merge(
    institutions[['InstitutionName', 'RegularSeasonWins', 'RegularSeasonLosses', 'RegularSeasonAverageAttendance', 'RegularSeasonAverageScore', 'InstitutionEnrollment_Male', 'InstitutionEnrollment_Female']],
    left_on='RegionWinner_East', right_on='InstitutionName', how='left'
).rename(columns={'RegularSeasonWins': 'RegionWinner_East_Wins', 'RegularSeasonLosses': 'RegionWinner_East_Losses',
                  'RegularSeasonAverageAttendance': 'RegionWinner_East_AvgAttendance', 'RegularSeasonAverageScore': 'RegionWinner_East_AvgScore',
                  'InstitutionEnrollment_Male': 'RegionWinner_East_Enrollment_Male', 'InstitutionEnrollment_Female': 'RegionWinner_East_Enrollment_Female'}).drop(columns='InstitutionName')

# Merge additional data for RegionWinner_West
bracket_test_simple = bracket_test_simple.merge(
    institutions[['InstitutionName', 'RegularSeasonWins', 'RegularSeasonLosses', 'RegularSeasonAverageAttendance', 'RegularSeasonAverageScore', 'InstitutionEnrollment_Male', 'InstitutionEnrollment_Female']],
    left_on='RegionWinner_West', right_on='InstitutionName', how='left'
).rename(columns={'RegularSeasonWins': 'RegionWinner_West_Wins', 'RegularSeasonLosses': 'RegionWinner_West_Losses',
                  'RegularSeasonAverageAttendance': 'RegionWinner_West_AvgAttendance', 'RegularSeasonAverageScore': 'RegionWinner_West_AvgScore',
                  'InstitutionEnrollment_Male': 'RegionWinner_West_Enrollment_Male', 'InstitutionEnrollment_Female': 'RegionWinner_West_Enrollment_Female'}).drop(columns='InstitutionName')

# Calculate win-loss ratio for RegionWinner_East
bracket_test_simple['RegionWinner_East_WinLossRatio'] = bracket_test_simple['RegionWinner_East_Wins'] / (bracket_test_simple['RegionWinner_East_Losses'] + 1e-9)

# Calculate win-loss ratio for RegionWinner_West
bracket_test_simple['RegionWinner_West_WinLossRatio'] = bracket_test_simple['RegionWinner_West_Wins'] / (bracket_test_simple['RegionWinner_West_Losses'] + 1e-9)

# Rearrange columns to group RegionWinner_East and RegionWinner_West columns together
bracket_test_simple = bracket_test_simple[['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_East_Wins', 'RegionWinner_East_Losses', 'RegionWinner_East_WinLossRatio',
                                           'RegionWinner_East_AvgAttendance', 'RegionWinner_East_AvgScore', 'RegionWinner_East_Enrollment_Male', 'RegionWinner_East_Enrollment_Female',
                                           'RegionWinner_West', 'RegionWinner_West_Wins', 'RegionWinner_West_Losses', 'RegionWinner_West_WinLossRatio',
                                           'RegionWinner_West_AvgAttendance', 'RegionWinner_West_AvgScore', 'RegionWinner_West_Enrollment_Male', 'RegionWinner_West_Enrollment_Female',
                                           'SemifinalWinner_East_West', 'BracketEntryId', 'NCAACustomerRecordCreated', 'BracketEntryCreatedDate', 'MonthsBetween']]

bracket_test_simple.set_index('BracketEntryId', inplace=True)
bracket_test_simple.head()

,CustomerDMACode,RegionWinner_East,RegionWinner_East_Wins,RegionWinner_East_Losses,RegionWinner_East_WinLossRatio,RegionWinner_East_AvgAttendance,RegionWinner_East_AvgScore,RegionWinner_East_Enrollment_Male,RegionWinner_East_Enrollment_Female,RegionWinner_West,...,RegionWinner_West_Losses,RegionWinner_West_WinLossRatio,RegionWinner_West_AvgAttendance,RegionWinner_West_AvgScore,RegionWinner_West_Enrollment_Male,RegionWinner_West_Enrollment_Female,SemifinalWinner_East_West,NCAACustomerRecordCreated,BracketEntryCreatedDate,MonthsBetween
BracketEntryId,,,,,,,,,,,,,,,,,,,,,
2074118,560,UConn,31,3,10.333333,14017.88,81.47,10645,11834,North Carolina,...,7,3.857143,15767.47,81.47,8068,12174,,2020-03-29,2024-03-19,48
2692634,820,UConn,31,3,10.333333,14017.88,81.47,10645,11834,North Carolina,...,7,3.857143,15767.47,81.47,8068,12174,,2024-04-02,2024-03-20,-1
1252684,529,Iowa St.,27,7,3.857143,12059.44,75.56,14070,11171,Arizona,...,8,3.125000,12118.18,87.94,13090,17292,,2021-12-08,2024-03-18,27
1950205,819,UConn,31,3,10.333333,14017.88,81.47,10645,11834,North Carolina,...,7,3.857143,15767.47,81.47,8068,12174,,2023-03-22,2024-03-19,12
2756293,504,UConn,31,3,10.333333,14017.88,81.47,10645,11834,North Carolina,...,7,3.857143,15767.47,81.47,8068,12174,,2024-02-20,2024-03-20,1


In [22]:
bracket_test_simple.to_csv('bracket_test_master.csv')


In [23]:
bracket_training_simple = pd.read_csv('bracket_training_master.csv')
bracket_training_simple.set_index('BracketEntryId', inplace=True)

bracket_test_simple = pd.read_csv('bracket_test_master.csv')
bracket_test_simple.set_index('BracketEntryId', inplace=True)


In [24]:
bracket_training_simple.columns

Index(['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_East_Wins',
       'RegionWinner_East_Losses', 'RegionWinner_East_WinLossRatio',
       'RegionWinner_East_AvgAttendance', 'RegionWinner_East_AvgScore',
       'RegionWinner_East_Enrollment_Male',
       'RegionWinner_East_Enrollment_Female', 'RegionWinner_West',
       'RegionWinner_West_Wins', 'RegionWinner_West_Losses',
       'RegionWinner_West_WinLossRatio', 'RegionWinner_West_AvgAttendance',
       'RegionWinner_West_AvgScore', 'RegionWinner_West_Enrollment_Male',
       'RegionWinner_West_Enrollment_Female', 'SemifinalWinner_East_West',
       'NCAACustomerRecordCreated', 'BracketEntryCreatedDate',
       'MonthsBetween'],
      dtype='object')

### Random Forest Model for Predicting Semifinal 1

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

# Define training data (from bracket_training_simple)
X_train = bracket_training_simple[['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_East_WinLossRatio', 'RegionWinner_East_AvgAttendance',
                                   'RegionWinner_East_AvgScore', 'RegionWinner_East_Enrollment_Male', 'RegionWinner_East_Enrollment_Female',
                                   'RegionWinner_West', 'RegionWinner_West_WinLossRatio', 'RegionWinner_West_AvgAttendance',
                                   'RegionWinner_West_AvgScore', 'RegionWinner_West_Enrollment_Male', 'RegionWinner_West_Enrollment_Female',
                                   'MonthsBetween']]
y_train = bracket_training_simple['SemifinalWinner_East_West']

# Define test data (from bracket_test_simple)
X_test = bracket_test_simple[['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_East_WinLossRatio', 'RegionWinner_East_AvgAttendance',
                              'RegionWinner_East_AvgScore', 'RegionWinner_East_Enrollment_Male', 'RegionWinner_East_Enrollment_Female',
                              'RegionWinner_West', 'RegionWinner_West_WinLossRatio', 'RegionWinner_West_AvgAttendance',
                              'RegionWinner_West_AvgScore', 'RegionWinner_West_Enrollment_Male', 'RegionWinner_West_Enrollment_Female',
                              'MonthsBetween']]

# Encode categorical features
cat_features = ['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_West']
label_encoders = {col: LabelEncoder().fit(X_train[col]) for col in cat_features}

for col, le in label_encoders.items():
    X_train[col] = le.transform(X_train[col])
    X_test[col] = le.transform(X_test[col])

# Set up the parameter grid for grid search
param_grid = {
    'n_estimators': [300],
    # 'max_depth': [10, 11, 12],
    'max_depth': [11],
    'min_samples_split': [5],
    # 'min_samples_leaf': [4, 5, 6]
    'min_samples_leaf': [5]
}

# Initialize the RandomForestClassifier
rf = RandomForestClassifier()

# Set up the grid search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring='accuracy', verbose=2, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best model
semifinal1_best_model = grid_search.best_estimator_

print(f"Best Parameters: {grid_search.best_params_}")

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END max_depth=11, min_samples_leaf=5, min_samples_split=5, n_estimators=300; total time=   9.6s
[CV] END max_depth=11, min_samples_leaf=5, min_samples_split=5, n_estimators=300; total time=   9.7s
[CV] END max_depth=11, min_samples_leaf=5, min_samples_split=5, n_estimators=300; total time=   9.7s
Best Parameters: {'max_depth': 11, 'min_samples_leaf': 5, 'min_samples_split': 5, 'n_estimators': 300}


In [26]:
bracket_test_simple['SemifinalWinner_East_West'] = semifinal1_best_model.predict(X_test).flatten()
bracket_test_simple

,CustomerDMACode,RegionWinner_East,RegionWinner_East_Wins,RegionWinner_East_Losses,RegionWinner_East_WinLossRatio,RegionWinner_East_AvgAttendance,RegionWinner_East_AvgScore,RegionWinner_East_Enrollment_Male,RegionWinner_East_Enrollment_Female,RegionWinner_West,...,RegionWinner_West_Losses,RegionWinner_West_WinLossRatio,RegionWinner_West_AvgAttendance,RegionWinner_West_AvgScore,RegionWinner_West_Enrollment_Male,RegionWinner_West_Enrollment_Female,SemifinalWinner_East_West,NCAACustomerRecordCreated,BracketEntryCreatedDate,MonthsBetween
BracketEntryId,,,,,,,,,,,,,,,,,,,,,
2074118,560,UConn,31,3,10.333333,14017.88,81.47,10645,11834,North Carolina,...,7,3.857143,15767.47,81.47,8068,12174,North Carolina,2020-03-29,2024-03-19,48
2692634,820,UConn,31,3,10.333333,14017.88,81.47,10645,11834,North Carolina,...,7,3.857143,15767.47,81.47,8068,12174,UConn,2024-04-02,2024-03-20,-1
1252684,529,Iowa St.,27,7,3.857143,12059.44,75.56,14070,11171,Arizona,...,8,3.125000,12118.18,87.94,13090,17292,Iowa St.,2021-12-08,2024-03-18,27
1950205,819,UConn,31,3,10.333333,14017.88,81.47,10645,11834,North Carolina,...,7,3.857143,15767.47,81.47,8068,12174,UConn,2023-03-22,2024-03-19,12
2756293,504,UConn,31,3,10.333333,14017.88,81.47,10645,11834,North Carolina,...,7,3.857143,15767.47,81.47,8068,12174,UConn,2024-02-20,2024-03-20,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2085292,789,UConn,31,3,10.333333,14017.88,81.47,10645,11834,Arizona,...,8,3.125000,12118.18,87.94,13090,17292,Arizona,2022-02-11,2024-03-19,25
615949,516,UConn,31,3,10.333333,14017.88,81.47,10645,11834,North Carolina,...,7,3.857143,15767.47,81.47,8068,12174,UConn,2021-03-19,2024-03-17,36
13727,862,BYU,23,10,2.300000,13481.27,81.85,13555,14698,Arizona,...,8,3.125000,12118.18,87.94,13090,17292,Arizona,2021-02-22,2024-02-21,36


## SemiFinal 2 Predictions

### Master Training Dataframe

In [27]:
# Select relevant columns
bracket_training_semifinal2 = bracket_training[['CustomerDMACode', 'RegionWinner_South', 'RegionWinner_Midwest', 
                                                'SemifinalWinner_South_Midwest', 'BracketEntryId', 'NCAACustomerRecordCreated', 
                                                'BracketEntryCreatedDate']]

# Process CustomerDMACode
bracket_training_semifinal2['CustomerDMACode'] = bracket_training_semifinal2['CustomerDMACode'].astype(str).str[:-2]
bracket_training_semifinal2['BracketEntryCreatedDate'] = bracket_training_semifinal2['BracketEntryCreatedDate'].astype(str).str.split(' ').str[0]

# Convert to datetime
bracket_training_semifinal2['BracketEntryCreatedDate'] = pd.to_datetime(bracket_training_semifinal2['BracketEntryCreatedDate'], infer_datetime_format=True, errors='coerce')
bracket_training_semifinal2['NCAACustomerRecordCreated'] = pd.to_datetime(bracket_training_semifinal2['NCAACustomerRecordCreated'], infer_datetime_format=True, errors='coerce')

# Calculate the number of months between the two dates
bracket_training_semifinal2['MonthsBetween'] = ((bracket_training_semifinal2['BracketEntryCreatedDate'].dt.year - bracket_training_semifinal2['NCAACustomerRecordCreated'].dt.year) * 12 +
                                        (bracket_training_semifinal2['BracketEntryCreatedDate'].dt.month - bracket_training_semifinal2['NCAACustomerRecordCreated'].dt.month))

# Merge additional data for RegionWinner_South
bracket_training_semifinal2 = bracket_training_semifinal2.merge(
    institutions[['InstitutionName', 'RegularSeasonWins', 'RegularSeasonLosses', 'RegularSeasonAverageAttendance', 
                  'RegularSeasonAverageScore', 'InstitutionEnrollment_Male', 'InstitutionEnrollment_Female']],
    left_on='RegionWinner_South', right_on='InstitutionName', how='left'
).rename(columns={'RegularSeasonWins': 'RegionWinner_South_Wins', 'RegularSeasonLosses': 'RegionWinner_South_Losses',
                  'RegularSeasonAverageAttendance': 'RegionWinner_South_AvgAttendance', 'RegularSeasonAverageScore': 'RegionWinner_South_AvgScore',
                  'InstitutionEnrollment_Male': 'RegionWinner_South_Enrollment_Male', 'InstitutionEnrollment_Female': 'RegionWinner_South_Enrollment_Female'}).drop(columns='InstitutionName')

# Merge additional data for RegionWinner_Midwest
bracket_training_semifinal2 = bracket_training_semifinal2.merge(
    institutions[['InstitutionName', 'RegularSeasonWins', 'RegularSeasonLosses', 'RegularSeasonAverageAttendance', 'RegularSeasonAverageScore', 'InstitutionEnrollment_Male', 'InstitutionEnrollment_Female']],
    left_on='RegionWinner_Midwest', right_on='InstitutionName', how='left'
).rename(columns={'RegularSeasonWins': 'RegionWinner_Midwest_Wins', 'RegularSeasonLosses': 'RegionWinner_Midwest_Losses',
                  'RegularSeasonAverageAttendance': 'RegionWinner_Midwest_AvgAttendance', 'RegularSeasonAverageScore': 'RegionWinner_Midwest_AvgScore',
                  'InstitutionEnrollment_Male': 'RegionWinner_Midwest_Enrollment_Male', 'InstitutionEnrollment_Female': 'RegionWinner_Midwest_Enrollment_Female'}).drop(columns='InstitutionName')

# Calculate win-loss ratio for RegionWinner_South
bracket_training_semifinal2['RegionWinner_South_WinLossRatio'] = bracket_training_semifinal2['RegionWinner_South_Wins'] / (bracket_training_semifinal2['RegionWinner_South_Losses'] + 1e-9)

# Calculate win-loss ratio for RegionWinner_Midwest
bracket_training_semifinal2['RegionWinner_Midwest_WinLossRatio'] = bracket_training_semifinal2['RegionWinner_Midwest_Wins'] / (bracket_training_semifinal2['RegionWinner_Midwest_Losses'] + 1e-9)

# Rearrange columns to group RegionWinner_South and RegionWinner_Midwest columns together
bracket_training_semifinal2 = bracket_training_semifinal2[['CustomerDMACode', 'RegionWinner_South', 'RegionWinner_South_Wins', 'RegionWinner_South_Losses', 'RegionWinner_South_WinLossRatio',
                                           'RegionWinner_South_AvgAttendance', 'RegionWinner_South_AvgScore', 'RegionWinner_South_Enrollment_Male', 'RegionWinner_South_Enrollment_Female',
                                           'RegionWinner_Midwest', 'RegionWinner_Midwest_Wins', 'RegionWinner_Midwest_Losses', 'RegionWinner_Midwest_WinLossRatio',
                                           'RegionWinner_Midwest_AvgAttendance', 'RegionWinner_Midwest_AvgScore', 'RegionWinner_Midwest_Enrollment_Male', 'RegionWinner_Midwest_Enrollment_Female',
                                           'SemifinalWinner_South_Midwest', 'BracketEntryId', 'NCAACustomerRecordCreated', 'BracketEntryCreatedDate', 'MonthsBetween']]

# Set BracketEntryId as index
bracket_training_semifinal2.set_index('BracketEntryId', inplace=True)

# Display first few rows
bracket_training_semifinal2.head()

,CustomerDMACode,RegionWinner_South,RegionWinner_South_Wins,RegionWinner_South_Losses,RegionWinner_South_WinLossRatio,RegionWinner_South_AvgAttendance,RegionWinner_South_AvgScore,RegionWinner_South_Enrollment_Male,RegionWinner_South_Enrollment_Female,RegionWinner_Midwest,...,RegionWinner_Midwest_Losses,RegionWinner_Midwest_WinLossRatio,RegionWinner_Midwest_AvgAttendance,RegionWinner_Midwest_AvgScore,RegionWinner_Midwest_Enrollment_Male,RegionWinner_Midwest_Enrollment_Female,SemifinalWinner_South_Midwest,NCAACustomerRecordCreated,BracketEntryCreatedDate,MonthsBetween
BracketEntryId,,,,,,,,,,,,,,,,,,,,,
1723503,698,James Madison,31,3,10.333333,4471.35,84.35,8342,12004,Tennessee,...,8,3.000000,16065.47,79.47,15269,18536,Tennessee,2021-12-25,2024-03-19,27
963479,563,Kentucky,23,9,2.555556,17427.94,89.44,9596,13127,Kansas,...,10,2.200000,13952.66,75.25,8859,9845,Kentucky,2021-04-02,2024-03-18,35
810038,511,Houston,30,4,7.500000,9347.35,73.03,18290,19653,Purdue,...,4,7.250000,13329.06,83.39,21670,16279,Purdue,2021-04-02,2024-03-18,35
3384825,641,NC State,22,14,1.571429,12147.11,76.36,11649,11794,Purdue,...,4,7.250000,13329.06,83.39,21670,16279,NC State,2023-11-16,2024-03-21,4
2828017,514,Marquette,25,9,2.777778,14084.65,78.29,3328,4200,Creighton,...,9,2.555556,13651.44,80.53,1789,2501,Marquette,2022-03-16,2024-03-20,24


### Master Test Dataframe

In [28]:
bracket_test_semifinal2 = bracket_test[['CustomerDMACode', 'RegionWinner_South', 'RegionWinner_Midwest', 'SemifinalWinner_South_Midwest', 'BracketEntryId', 'NCAACustomerRecordCreated', 'BracketEntryCreatedDate']]
bracket_test_semifinal2['CustomerDMACode'] = bracket_test_semifinal2['CustomerDMACode'].astype(str).str[:-2]
bracket_test_semifinal2['BracketEntryCreatedDate'] = bracket_test_semifinal2['BracketEntryCreatedDate'].astype(str).str.split(' ').str[0]

# Convert to datetime
bracket_test_semifinal2['BracketEntryCreatedDate'] = pd.to_datetime(bracket_test_semifinal2['BracketEntryCreatedDate'], infer_datetime_format=True, errors='coerce')
bracket_test_semifinal2['NCAACustomerRecordCreated'] = pd.to_datetime(bracket_test_semifinal2['NCAACustomerRecordCreated'], infer_datetime_format=True, errors='coerce')

# Calculate the number of months between the two dates
bracket_test_semifinal2['MonthsBetween'] = ((bracket_test_semifinal2['BracketEntryCreatedDate'].dt.year - bracket_test_semifinal2['NCAACustomerRecordCreated'].dt.year) * 12 +
                                        (bracket_test_semifinal2['BracketEntryCreatedDate'].dt.month - bracket_test_semifinal2['NCAACustomerRecordCreated'].dt.month))

# Merge additional data for RegionWinner_South
bracket_test_semifinal2 = bracket_test_semifinal2.merge(
    institutions[['InstitutionName', 'RegularSeasonWins', 'RegularSeasonLosses', 'RegularSeasonAverageAttendance', 'RegularSeasonAverageScore', 'InstitutionEnrollment_Male', 'InstitutionEnrollment_Female']],
    left_on='RegionWinner_South', right_on='InstitutionName', how='left'
).rename(columns={'RegularSeasonWins': 'RegionWinner_South_Wins', 'RegularSeasonLosses': 'RegionWinner_South_Losses',
                  'RegularSeasonAverageAttendance': 'RegionWinner_South_AvgAttendance', 'RegularSeasonAverageScore': 'RegionWinner_South_AvgScore',
                  'InstitutionEnrollment_Male': 'RegionWinner_South_Enrollment_Male', 'InstitutionEnrollment_Female': 'RegionWinner_South_Enrollment_Female'}).drop(columns='InstitutionName')

# Merge additional data for RegionWinner_Midwest
bracket_test_semifinal2 = bracket_test_semifinal2.merge(
    institutions[['InstitutionName', 'RegularSeasonWins', 'RegularSeasonLosses', 'RegularSeasonAverageAttendance', 'RegularSeasonAverageScore', 'InstitutionEnrollment_Male', 'InstitutionEnrollment_Female']],
    left_on='RegionWinner_Midwest', right_on='InstitutionName', how='left'
).rename(columns={'RegularSeasonWins': 'RegionWinner_Midwest_Wins', 'RegularSeasonLosses': 'RegionWinner_Midwest_Losses',
                  'RegularSeasonAverageAttendance': 'RegionWinner_Midwest_AvgAttendance', 'RegularSeasonAverageScore': 'RegionWinner_Midwest_AvgScore',
                  'InstitutionEnrollment_Male': 'RegionWinner_Midwest_Enrollment_Male', 'InstitutionEnrollment_Female': 'RegionWinner_Midwest_Enrollment_Female'}).drop(columns='InstitutionName')

# Calculate win-loss ratio for RegionWinner_South
bracket_test_semifinal2['RegionWinner_South_WinLossRatio'] = bracket_test_semifinal2['RegionWinner_South_Wins'] / (bracket_test_semifinal2['RegionWinner_South_Losses'] + 1e-9)

# Calculate win-loss ratio for RegionWinner_Midwest
bracket_test_semifinal2['RegionWinner_Midwest_WinLossRatio'] = bracket_test_semifinal2['RegionWinner_Midwest_Wins'] / (bracket_test_semifinal2['RegionWinner_Midwest_Losses'] + 1e-9)

# Rearrange columns to group RegionWinner_South and RegionWinner_Midwest columns together
bracket_test_semifinal2 = bracket_test_semifinal2[['CustomerDMACode', 'RegionWinner_South', 'RegionWinner_South_Wins', 'RegionWinner_South_Losses', 'RegionWinner_South_WinLossRatio',
                                           'RegionWinner_South_AvgAttendance', 'RegionWinner_South_AvgScore', 'RegionWinner_South_Enrollment_Male', 'RegionWinner_South_Enrollment_Female',
                                           'RegionWinner_Midwest', 'RegionWinner_Midwest_Wins', 'RegionWinner_Midwest_Losses', 'RegionWinner_Midwest_WinLossRatio',
                                           'RegionWinner_Midwest_AvgAttendance', 'RegionWinner_Midwest_AvgScore', 'RegionWinner_Midwest_Enrollment_Male', 'RegionWinner_Midwest_Enrollment_Female',
                                           'SemifinalWinner_South_Midwest', 'BracketEntryId', 'NCAACustomerRecordCreated', 'BracketEntryCreatedDate', 'MonthsBetween']]

# Set BracketEntryId as index
bracket_test_semifinal2.set_index('BracketEntryId', inplace=True)

# Display first few rows
bracket_test_semifinal2.head()

,CustomerDMACode,RegionWinner_South,RegionWinner_South_Wins,RegionWinner_South_Losses,RegionWinner_South_WinLossRatio,RegionWinner_South_AvgAttendance,RegionWinner_South_AvgScore,RegionWinner_South_Enrollment_Male,RegionWinner_South_Enrollment_Female,RegionWinner_Midwest,...,RegionWinner_Midwest_Losses,RegionWinner_Midwest_WinLossRatio,RegionWinner_Midwest_AvgAttendance,RegionWinner_Midwest_AvgScore,RegionWinner_Midwest_Enrollment_Male,RegionWinner_Midwest_Enrollment_Female,SemifinalWinner_South_Midwest,NCAACustomerRecordCreated,BracketEntryCreatedDate,MonthsBetween
BracketEntryId,,,,,,,,,,,,,,,,,,,,,
2074118,560,Houston,30,4,7.500000,9347.35,73.03,18290,19653,Purdue,...,4,7.250000,13329.06,83.39,21670,16279,,2020-03-29,2024-03-19,48
2692634,820,Duke,24,8,3.000000,11475.81,79.84,3137,3448,Kansas,...,10,2.200000,13952.66,75.25,8859,9845,,2024-04-02,2024-03-20,-1
1252684,529,Kentucky,23,9,2.555556,17427.94,89.44,9596,13127,Creighton,...,9,2.555556,13651.44,80.53,1789,2501,,2021-12-08,2024-03-18,27
1950205,819,Houston,30,4,7.500000,9347.35,73.03,18290,19653,Purdue,...,4,7.250000,13329.06,83.39,21670,16279,,2023-03-22,2024-03-19,12
2756293,504,Marquette,25,9,2.777778,14084.65,78.29,3328,4200,Creighton,...,9,2.555556,13651.44,80.53,1789,2501,,2024-02-20,2024-03-20,1


In [29]:
bracket_test_semifinal2.columns

Index(['CustomerDMACode', 'RegionWinner_South', 'RegionWinner_South_Wins',
       'RegionWinner_South_Losses', 'RegionWinner_South_WinLossRatio',
       'RegionWinner_South_AvgAttendance', 'RegionWinner_South_AvgScore',
       'RegionWinner_South_Enrollment_Male',
       'RegionWinner_South_Enrollment_Female', 'RegionWinner_Midwest',
       'RegionWinner_Midwest_Wins', 'RegionWinner_Midwest_Losses',
       'RegionWinner_Midwest_WinLossRatio',
       'RegionWinner_Midwest_AvgAttendance', 'RegionWinner_Midwest_AvgScore',
       'RegionWinner_Midwest_Enrollment_Male',
       'RegionWinner_Midwest_Enrollment_Female',
       'SemifinalWinner_South_Midwest', 'NCAACustomerRecordCreated',
       'BracketEntryCreatedDate', 'MonthsBetween'],
      dtype='object')

### Random Forest Model for Semifinal 2

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

# Define training data (from bracket_training_semifinal2)
X_train = bracket_training_semifinal2[['CustomerDMACode', 'RegionWinner_South', 'RegionWinner_South_WinLossRatio', 'RegionWinner_South_AvgAttendance',
                                       'RegionWinner_South_AvgScore', 'RegionWinner_South_Enrollment_Male', 'RegionWinner_South_Enrollment_Female',
                                       'RegionWinner_Midwest', 'RegionWinner_Midwest_WinLossRatio', 'RegionWinner_Midwest_AvgAttendance',
                                       'RegionWinner_Midwest_AvgScore', 'RegionWinner_Midwest_Enrollment_Male', 'RegionWinner_Midwest_Enrollment_Female',
                                       'MonthsBetween']]
y_train = bracket_training_semifinal2['SemifinalWinner_South_Midwest']

# Define test data (from bracket_test_semifinal2)
X_test = bracket_test_semifinal2[['CustomerDMACode', 'RegionWinner_South', 'RegionWinner_South_WinLossRatio', 'RegionWinner_South_AvgAttendance',
                                  'RegionWinner_South_AvgScore', 'RegionWinner_South_Enrollment_Male', 'RegionWinner_South_Enrollment_Female',
                                  'RegionWinner_Midwest', 'RegionWinner_Midwest_WinLossRatio', 'RegionWinner_Midwest_AvgAttendance',
                                  'RegionWinner_Midwest_AvgScore', 'RegionWinner_Midwest_Enrollment_Male', 'RegionWinner_Midwest_Enrollment_Female',
                                  'MonthsBetween']]

# Encode categorical features
cat_features = ['CustomerDMACode', 'RegionWinner_South', 'RegionWinner_Midwest']
label_encoders = {col: LabelEncoder().fit(X_train[col]) for col in cat_features}

for col, le in label_encoders.items():
    X_train[col] = le.transform(X_train[col])
    X_test[col] = le.transform(X_test[col])

# Set up the parameter grid for grid search
param_grid = {
    # 'n_estimators': [400, 500],
    'n_estimators': [400],
    # 'max_depth': [8, 9, 10],
    'max_depth': [10],
    # 'min_samples_split': [4, 5, 6],
    'min_samples_split': [5],
    # 'min_samples_leaf': [6, 7, 8]
    'min_samples_leaf': [6]
}

# Initialize the RandomForestClassifier
rf = RandomForestClassifier()

# Set up the grid search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring='accuracy', verbose=2, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

print(f"Best Parameters: {grid_search.best_params_}")

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END max_depth=10, min_samples_leaf=6, min_samples_split=5, n_estimators=400; total time=  12.7s
[CV] END max_depth=10, min_samples_leaf=6, min_samples_split=5, n_estimators=400; total time=  12.9s
[CV] END max_depth=10, min_samples_leaf=6, min_samples_split=5, n_estimators=400; total time=  12.9s
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 6, 'min_samples_split': 5, 'n_estimators': 400}


In [36]:
bracket_test_semifinal2['SemifinalWinner_South_Midwest'] = best_model.predict(X_test).flatten()
bracket_test_semifinal2.head()

,CustomerDMACode,RegionWinner_South,RegionWinner_South_Wins,RegionWinner_South_Losses,RegionWinner_South_WinLossRatio,RegionWinner_South_AvgAttendance,RegionWinner_South_AvgScore,RegionWinner_South_Enrollment_Male,RegionWinner_South_Enrollment_Female,RegionWinner_Midwest,RegionWinner_Midwest_Wins,RegionWinner_Midwest_Losses,RegionWinner_Midwest_WinLossRatio,RegionWinner_Midwest_AvgAttendance,RegionWinner_Midwest_AvgScore,RegionWinner_Midwest_Enrollment_Male,RegionWinner_Midwest_Enrollment_Female,SemifinalWinner_South_Midwest,NCAACustomerRecordCreated,BracketEntryCreatedDate,MonthsBetween
BracketEntryId,,,,,,,,,,,,,,,,,,,,,
2074118,560,Houston,30,4,7.500000,9347.35,73.03,18290,19653,Purdue,29,4,7.250000,13329.06,83.39,21670,16279,Houston,2020-03-29,2024-03-19,48
2692634,820,Duke,24,8,3.000000,11475.81,79.84,3137,3448,Kansas,22,10,2.200000,13952.66,75.25,8859,9845,Duke,2024-04-02,2024-03-20,-1
1252684,529,Kentucky,23,9,2.555556,17427.94,89.44,9596,13127,Creighton,23,9,2.555556,13651.44,80.53,1789,2501,Kentucky,2021-12-08,2024-03-18,27
1950205,819,Houston,30,4,7.500000,9347.35,73.03,18290,19653,Purdue,29,4,7.250000,13329.06,83.39,21670,16279,Houston,2023-03-22,2024-03-19,12
2756293,504,Marquette,25,9,2.777778,14084.65,78.29,3328,4200,Creighton,23,9,2.555556,13651.44,80.53,1789,2501,Marquette,2024-02-20,2024-03-20,1


## National Champion Predictions

### Master Training Dataframe

In [32]:
# Select relevant columns
finals_training_df = bracket_training[['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_West', 'RegionWinner_South', 
                                       'RegionWinner_Midwest', 'SemifinalWinner_East_West', 'SemifinalWinner_South_Midwest', 
                                       'NationalChampion', 'BracketEntryId', 'NCAACustomerRecordCreated', 'BracketEntryCreatedDate']]

# Process CustomerDMACode
finals_training_df['CustomerDMACode'] = finals_training_df['CustomerDMACode'].astype(str).str[:-2]
finals_training_df['BracketEntryCreatedDate'] = finals_training_df['BracketEntryCreatedDate'].astype(str).str.split(' ').str[0]

# Convert to datetime
finals_training_df['BracketEntryCreatedDate'] = pd.to_datetime(finals_training_df['BracketEntryCreatedDate'], infer_datetime_format=True, errors='coerce')
finals_training_df['NCAACustomerRecordCreated'] = pd.to_datetime(finals_training_df['NCAACustomerRecordCreated'], infer_datetime_format=True, errors='coerce')

# Calculate the number of months between the two dates
finals_training_df['MonthsBetween'] = ((finals_training_df['BracketEntryCreatedDate'].dt.year - finals_training_df['NCAACustomerRecordCreated'].dt.year) * 12 +
                                        (finals_training_df['BracketEntryCreatedDate'].dt.month - finals_training_df['NCAACustomerRecordCreated'].dt.month))

# ✅ Merge additional data for **Region Winners**
for region in ['East', 'West', 'South', 'Midwest']:
    finals_training_df = finals_training_df.merge(
        institutions[['InstitutionName', 'RegularSeasonWins', 'RegularSeasonLosses', 'RegularSeasonAverageAttendance', 
                      'RegularSeasonAverageScore', 'InstitutionEnrollment_Male', 'InstitutionEnrollment_Female']],
        left_on=f'RegionWinner_{region}', right_on='InstitutionName', how='left'
    ).rename(columns={
        'RegularSeasonWins': f'RegionWinner_{region}_Wins',
        'RegularSeasonLosses': f'RegionWinner_{region}_Losses',
        'RegularSeasonAverageAttendance': f'RegionWinner_{region}_AvgAttendance',
        'RegularSeasonAverageScore': f'RegionWinner_{region}_AvgScore',
        'InstitutionEnrollment_Male': f'RegionWinner_{region}_Enrollment_Male',
        'InstitutionEnrollment_Female': f'RegionWinner_{region}_Enrollment_Female'
    }).drop(columns='InstitutionName')

# ✅ Merge additional data for **Semifinal Winners**
for semifinal in ['SemifinalWinner_East_West', 'SemifinalWinner_South_Midwest']:
    finals_training_df = finals_training_df.merge(
        institutions[['InstitutionName', 'RegularSeasonWins', 'RegularSeasonLosses', 'RegularSeasonAverageAttendance', 
                      'RegularSeasonAverageScore', 'InstitutionEnrollment_Male', 'InstitutionEnrollment_Female']],
        left_on=semifinal, right_on='InstitutionName', how='left'
    ).rename(columns={
        'RegularSeasonWins': f'{semifinal}_Wins',
        'RegularSeasonLosses': f'{semifinal}_Losses',
        'RegularSeasonAverageAttendance': f'{semifinal}_AvgAttendance',
        'RegularSeasonAverageScore': f'{semifinal}_AvgScore',
        'InstitutionEnrollment_Male': f'{semifinal}_Enrollment_Male',
        'InstitutionEnrollment_Female': f'{semifinal}_Enrollment_Female'
    }).drop(columns='InstitutionName')

# ✅ Calculate **Win-Loss Ratios** for region winners and semifinal winners
for entity in ['RegionWinner_East', 'RegionWinner_West', 'RegionWinner_South', 'RegionWinner_Midwest',
               'SemifinalWinner_East_West', 'SemifinalWinner_South_Midwest']:
    finals_training_df[f'{entity}_WinLossRatio'] = finals_training_df[f'{entity}_Wins'] / (finals_training_df[f'{entity}_Losses'] + 1e-9)

# ✅ Rearrange columns for better organization
finals_training_df = finals_training_df[['CustomerDMACode', 
                                         # Region winners and their stats
                                         'RegionWinner_East', 'RegionWinner_East_Wins', 'RegionWinner_East_Losses', 'RegionWinner_East_WinLossRatio', 
                                         'RegionWinner_East_AvgAttendance', 'RegionWinner_East_AvgScore', 'RegionWinner_East_Enrollment_Male', 'RegionWinner_East_Enrollment_Female',
                                         'RegionWinner_West', 'RegionWinner_West_Wins', 'RegionWinner_West_Losses', 'RegionWinner_West_WinLossRatio', 
                                         'RegionWinner_West_AvgAttendance', 'RegionWinner_West_AvgScore', 'RegionWinner_West_Enrollment_Male', 'RegionWinner_West_Enrollment_Female',
                                         'RegionWinner_South', 'RegionWinner_South_Wins', 'RegionWinner_South_Losses', 'RegionWinner_South_WinLossRatio', 
                                         'RegionWinner_South_AvgAttendance', 'RegionWinner_South_AvgScore', 'RegionWinner_South_Enrollment_Male', 'RegionWinner_South_Enrollment_Female',
                                         'RegionWinner_Midwest', 'RegionWinner_Midwest_Wins', 'RegionWinner_Midwest_Losses', 'RegionWinner_Midwest_WinLossRatio', 
                                         'RegionWinner_Midwest_AvgAttendance', 'RegionWinner_Midwest_AvgScore', 'RegionWinner_Midwest_Enrollment_Male', 'RegionWinner_Midwest_Enrollment_Female',
                                         # Semifinal winners and their stats
                                         'SemifinalWinner_East_West', 'SemifinalWinner_East_West_Wins', 'SemifinalWinner_East_West_Losses', 'SemifinalWinner_East_West_WinLossRatio',
                                         'SemifinalWinner_East_West_AvgAttendance', 'SemifinalWinner_East_West_AvgScore', 'SemifinalWinner_East_West_Enrollment_Male', 'SemifinalWinner_East_West_Enrollment_Female',
                                         'SemifinalWinner_South_Midwest', 'SemifinalWinner_South_Midwest_Wins', 'SemifinalWinner_South_Midwest_Losses', 'SemifinalWinner_South_Midwest_WinLossRatio',
                                         'SemifinalWinner_South_Midwest_AvgAttendance', 'SemifinalWinner_South_Midwest_AvgScore', 'SemifinalWinner_South_Midwest_Enrollment_Male', 'SemifinalWinner_South_Midwest_Enrollment_Female',
                                         # Additional features
                                         'NationalChampion', 'BracketEntryId', 'NCAACustomerRecordCreated', 'BracketEntryCreatedDate', 'MonthsBetween']]

# Set BracketEntryId as index
finals_training_df.set_index('BracketEntryId', inplace=True)

# Display first few rows
finals_training_df.head()

,CustomerDMACode,RegionWinner_East,RegionWinner_East_Wins,RegionWinner_East_Losses,RegionWinner_East_WinLossRatio,RegionWinner_East_AvgAttendance,RegionWinner_East_AvgScore,RegionWinner_East_Enrollment_Male,RegionWinner_East_Enrollment_Female,RegionWinner_West,...,SemifinalWinner_South_Midwest_Losses,SemifinalWinner_South_Midwest_WinLossRatio,SemifinalWinner_South_Midwest_AvgAttendance,SemifinalWinner_South_Midwest_AvgScore,SemifinalWinner_South_Midwest_Enrollment_Male,SemifinalWinner_South_Midwest_Enrollment_Female,NationalChampion,NCAACustomerRecordCreated,BracketEntryCreatedDate,MonthsBetween
BracketEntryId,,,,,,,,,,,,,,,,,,,,,
1723503,698,UConn,31,3,10.333333,14017.88,81.47,10645,11834,Arizona,...,8,3.000000,16065.47,79.47,15269,18536,Tennessee,2021-12-25,2024-03-19,27
963479,563,UConn,31,3,10.333333,14017.88,81.47,10645,11834,Baylor,...,9,2.555556,17427.94,89.44,9596,13127,UConn,2021-04-02,2024-03-18,35
810038,511,UConn,31,3,10.333333,14017.88,81.47,10645,11834,Baylor,...,4,7.250000,13329.06,83.39,21670,16279,Purdue,2021-04-02,2024-03-18,35
3384825,641,Iowa St.,27,7,3.857143,12059.44,75.56,14070,11171,Saint Mary's,...,14,1.571429,12147.11,76.36,11649,11794,NC State,2023-11-16,2024-03-21,4
2828017,514,Auburn,27,7,3.857143,10182.91,83.32,12695,12684,North Carolina,...,9,2.777778,14084.65,78.29,3328,4200,North Carolina,2022-03-16,2024-03-20,24


### Master Test Dataframe

In [42]:
# Option 1: Set display options in pandas
import pandas as pd
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows
pd.set_option('display.width', None)        # Don't truncate width
pd.set_option('display.max_colwidth', None) # Don't truncate column content

# Select relevant columns from bracket_test
finals_test_df = bracket_test[['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_West', 'RegionWinner_South', 
                               'RegionWinner_Midwest', 'BracketEntryId', 'NCAACustomerRecordCreated', 'BracketEntryCreatedDate']]

# Process CustomerDMACode
finals_test_df['CustomerDMACode'] = finals_test_df['CustomerDMACode'].astype(str).str[:-2]
finals_test_df['BracketEntryCreatedDate'] = finals_test_df['BracketEntryCreatedDate'].astype(str).str.split(' ').str[0]

# Convert to datetime
finals_test_df['BracketEntryCreatedDate'] = pd.to_datetime(finals_test_df['BracketEntryCreatedDate'], infer_datetime_format=True, errors='coerce')
finals_test_df['NCAACustomerRecordCreated'] = pd.to_datetime(finals_test_df['NCAACustomerRecordCreated'], infer_datetime_format=True, errors='coerce')

# Calculate the number of months between the two dates
finals_test_df['MonthsBetween'] = ((finals_test_df['BracketEntryCreatedDate'].dt.year - finals_test_df['NCAACustomerRecordCreated'].dt.year) * 12 +
                                   (finals_test_df['BracketEntryCreatedDate'].dt.month - finals_test_df['NCAACustomerRecordCreated'].dt.month))

# ✅ Merge **Predicted Semifinal Winners** (These come from previous models)
finals_test_df = finals_test_df.merge(bracket_test_simple[['SemifinalWinner_East_West']], on='BracketEntryId', how='left')
finals_test_df = finals_test_df.merge(bracket_test_semifinal2[['SemifinalWinner_South_Midwest']], on='BracketEntryId', how='left')

# ✅ Merge additional data for **Region Winners**
for region in ['East', 'West', 'South', 'Midwest']:
    finals_test_df = finals_test_df.merge(
        institutions[['InstitutionName', 'RegularSeasonWins', 'RegularSeasonLosses', 'RegularSeasonAverageAttendance', 
                      'RegularSeasonAverageScore', 'InstitutionEnrollment_Male', 'InstitutionEnrollment_Female']],
        left_on=f'RegionWinner_{region}', right_on='InstitutionName', how='left'
    ).rename(columns={
        'RegularSeasonWins': f'RegionWinner_{region}_Wins',
        'RegularSeasonLosses': f'RegionWinner_{region}_Losses',
        'RegularSeasonAverageAttendance': f'RegionWinner_{region}_AvgAttendance',
        'RegularSeasonAverageScore': f'RegionWinner_{region}_AvgScore',
        'InstitutionEnrollment_Male': f'RegionWinner_{region}_Enrollment_Male',
        'InstitutionEnrollment_Female': f'RegionWinner_{region}_Enrollment_Female'
    }).drop(columns='InstitutionName')

# ✅ Merge additional data for **Semifinal Winners**
for semifinal in ['SemifinalWinner_East_West', 'SemifinalWinner_South_Midwest']:
    finals_test_df = finals_test_df.merge(
        institutions[['InstitutionName', 'RegularSeasonWins', 'RegularSeasonLosses', 'RegularSeasonAverageAttendance', 
                      'RegularSeasonAverageScore', 'InstitutionEnrollment_Male', 'InstitutionEnrollment_Female']],
        left_on=semifinal, right_on='InstitutionName', how='left'
    ).rename(columns={
        'RegularSeasonWins': f'{semifinal}_Wins',
        'RegularSeasonLosses': f'{semifinal}_Losses',
        'RegularSeasonAverageAttendance': f'{semifinal}_AvgAttendance',
        'RegularSeasonAverageScore': f'{semifinal}_AvgScore',
        'InstitutionEnrollment_Male': f'{semifinal}_Enrollment_Male',
        'InstitutionEnrollment_Female': f'{semifinal}_Enrollment_Female'
    }).drop(columns='InstitutionName')

# ✅ Calculate **Win-Loss Ratios** for region winners and semifinal winners
for entity in ['RegionWinner_East', 'RegionWinner_West', 'RegionWinner_South', 'RegionWinner_Midwest',
               'SemifinalWinner_East_West', 'SemifinalWinner_South_Midwest']:
    finals_test_df[f'{entity}_WinLossRatio'] = finals_test_df[f'{entity}_Wins'] / (finals_test_df[f'{entity}_Losses'] + 1e-9)

# ✅ Rearrange columns for better organization
finals_test_df = finals_test_df[['CustomerDMACode', 
                                 # Region winners and their stats
                                 'RegionWinner_East', 'RegionWinner_East_Wins', 'RegionWinner_East_Losses', 'RegionWinner_East_WinLossRatio', 
                                 'RegionWinner_East_AvgAttendance', 'RegionWinner_East_AvgScore', 'RegionWinner_East_Enrollment_Male', 'RegionWinner_East_Enrollment_Female',
                                 'RegionWinner_West', 'RegionWinner_West_Wins', 'RegionWinner_West_Losses', 'RegionWinner_West_WinLossRatio', 
                                 'RegionWinner_West_AvgAttendance', 'RegionWinner_West_AvgScore', 'RegionWinner_West_Enrollment_Male', 'RegionWinner_West_Enrollment_Female',
                                 'RegionWinner_South', 'RegionWinner_South_Wins', 'RegionWinner_South_Losses', 'RegionWinner_South_WinLossRatio', 
                                 'RegionWinner_South_AvgAttendance', 'RegionWinner_South_AvgScore', 'RegionWinner_South_Enrollment_Male', 'RegionWinner_South_Enrollment_Female',
                                 'RegionWinner_Midwest', 'RegionWinner_Midwest_Wins', 'RegionWinner_Midwest_Losses', 'RegionWinner_Midwest_WinLossRatio', 
                                 'RegionWinner_Midwest_AvgAttendance', 'RegionWinner_Midwest_AvgScore', 'RegionWinner_Midwest_Enrollment_Male', 'RegionWinner_Midwest_Enrollment_Female',
                                 # Semifinal winners and their stats
                                 'SemifinalWinner_East_West', 'SemifinalWinner_East_West_Wins', 'SemifinalWinner_East_West_Losses', 'SemifinalWinner_East_West_WinLossRatio',
                                 'SemifinalWinner_East_West_AvgAttendance', 'SemifinalWinner_East_West_AvgScore', 'SemifinalWinner_East_West_Enrollment_Male', 'SemifinalWinner_East_West_Enrollment_Female',
                                 'SemifinalWinner_South_Midwest', 'SemifinalWinner_South_Midwest_Wins', 'SemifinalWinner_South_Midwest_Losses', 'SemifinalWinner_South_Midwest_WinLossRatio',
                                 'SemifinalWinner_South_Midwest_AvgAttendance', 'SemifinalWinner_South_Midwest_AvgScore', 'SemifinalWinner_South_Midwest_Enrollment_Male', 'SemifinalWinner_South_Midwest_Enrollment_Female',
                                 # Additional features
                                'BracketEntryId', 'NCAACustomerRecordCreated', 'BracketEntryCreatedDate', 'MonthsBetween']]

# Set BracketEntryId as index
finals_test_df.set_index('BracketEntryId', inplace=True)

finals_test_df['NationalChampion'] = ""

# Display first few rows
finals_test_df.head()

,CustomerDMACode,RegionWinner_East,RegionWinner_East_Wins,RegionWinner_East_Losses,RegionWinner_East_WinLossRatio,RegionWinner_East_AvgAttendance,RegionWinner_East_AvgScore,RegionWinner_East_Enrollment_Male,RegionWinner_East_Enrollment_Female,RegionWinner_West,RegionWinner_West_Wins,RegionWinner_West_Losses,RegionWinner_West_WinLossRatio,RegionWinner_West_AvgAttendance,RegionWinner_West_AvgScore,RegionWinner_West_Enrollment_Male,RegionWinner_West_Enrollment_Female,RegionWinner_South,RegionWinner_South_Wins,RegionWinner_South_Losses,RegionWinner_South_WinLossRatio,RegionWinner_South_AvgAttendance,RegionWinner_South_AvgScore,RegionWinner_South_Enrollment_Male,RegionWinner_South_Enrollment_Female,RegionWinner_Midwest,RegionWinner_Midwest_Wins,RegionWinner_Midwest_Losses,RegionWinner_Midwest_WinLossRatio,RegionWinner_Midwest_AvgAttendance,RegionWinner_Midwest_AvgScore,RegionWinner_Midwest_Enrollment_Male,RegionWinner_Midwest_Enrollment_Female,SemifinalWinner_East_West,SemifinalWinner_East_West_Wins,SemifinalWinner_East_West_Losses,SemifinalWinner_East_West_WinLossRatio,SemifinalWinner_East_West_AvgAttendance,SemifinalWinner_East_West_AvgScore,SemifinalWinner_East_West_Enrollment_Male,SemifinalWinner_East_West_Enrollment_Female,SemifinalWinner_South_Midwest,SemifinalWinner_South_Midwest_Wins,SemifinalWinner_South_Midwest_Losses,SemifinalWinner_South_Midwest_WinLossRatio,SemifinalWinner_South_Midwest_AvgAttendance,SemifinalWinner_South_Midwest_AvgScore,SemifinalWinner_South_Midwest_Enrollment_Male,SemifinalWinner_South_Midwest_Enrollment_Female,NCAACustomerRecordCreated,BracketEntryCreatedDate,MonthsBetween,NationalChampion
BracketEntryId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2074118,560,UConn,31,3,10.333333,14017.88,81.47,10645,11834,North Carolina,27,7,3.857143,15767.47,81.47,8068,12174,Houston,30,4,7.500000,9347.35,73.03,18290,19653,Purdue,29,4,7.250000,13329.06,83.39,21670,16279,North Carolina,27,7,3.857143,15767.47,81.47,8068,12174,Houston,30,4,7.500000,9347.35,73.03,18290,19653,2020-03-29,2024-03-19,48,
2692634,820,UConn,31,3,10.333333,14017.88,81.47,10645,11834,North Carolina,27,7,3.857143,15767.47,81.47,8068,12174,Duke,24,8,3.000000,11475.81,79.84,3137,3448,Kansas,22,10,2.200000,13952.66,75.25,8859,9845,UConn,31,3,10.333333,14017.88,81.47,10645,11834,Duke,24,8,3.000000,11475.81,79.84,3137,3448,2024-04-02,2024-03-20,-1,
1252684,529,Iowa St.,27,7,3.857143,12059.44,75.56,14070,11171,Arizona,25,8,3.125000,12118.18,87.94,13090,17292,Kentucky,23,9,2.555556,17427.94,89.44,9596,13127,Creighton,23,9,2.555556,13651.44,80.53,1789,2501,Iowa St.,27,7,3.857143,12059.44,75.56,14070,11171,Kentucky,23,9,2.555556,17427.94,89.44,9596,13127,2021-12-08,2024-03-18,27,
1950205,819,UConn,31,3,10.333333,14017.88,81.47,10645,11834,North Carolina,27,7,3.857143,15767.47,81.47,8068,12174,Houston,30,4,7.500000,9347.35,73.03,18290,19653,Purdue,29,4,7.250000,13329.06,83.39,21670,16279,UConn,31,3,10.333333,14017.88,81.47,10645,11834,Houston,30,4,7.500000,9347.35,73.03,18290,19653,2023-03-22,2024-03-19,12,
2756293,504,UConn,31,3,10.333333,14017.88,81.47,10645,11834,North Carolina,27,7,3.857143,15767.47,81.47,8068,12174,Marquette,25,9,2.777778,14084.65,78.29,3328,4200,Creighton,23,9,2.555556,13651.44,80.53,1789,2501,UConn,31,3,10.333333,14017.88,81.47,10645,11834,Marquette,25,9,2.777778,14084.65,78.29,3328,4200,2024-02-20,2024-03-20,1,


### Random forest model

In [51]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV

# Define training data (from finals_training_df)
X_train = finals_training_df[['CustomerDMACode', 
                              # Semifinal winners and their stats
                              'SemifinalWinner_East_West', 'SemifinalWinner_East_West_WinLossRatio', 'SemifinalWinner_East_West_AvgAttendance',
                              'SemifinalWinner_East_West_AvgScore', 'SemifinalWinner_East_West_Enrollment_Male', 'SemifinalWinner_East_West_Enrollment_Female',
                              'SemifinalWinner_South_Midwest', 'SemifinalWinner_South_Midwest_WinLossRatio', 'SemifinalWinner_South_Midwest_AvgAttendance',
                              'SemifinalWinner_South_Midwest_AvgScore', 'SemifinalWinner_South_Midwest_Enrollment_Male', 'SemifinalWinner_South_Midwest_Enrollment_Female',
                              # Region winners and their stats
                              'RegionWinner_East', 'RegionWinner_East_WinLossRatio', 'RegionWinner_East_AvgAttendance',
                              'RegionWinner_East_AvgScore', 'RegionWinner_East_Enrollment_Male', 'RegionWinner_East_Enrollment_Female',
                              'RegionWinner_West', 'RegionWinner_West_WinLossRatio', 'RegionWinner_West_AvgAttendance',
                              'RegionWinner_West_AvgScore', 'RegionWinner_West_Enrollment_Male', 'RegionWinner_West_Enrollment_Female',
                              'RegionWinner_South', 'RegionWinner_South_WinLossRatio', 'RegionWinner_South_AvgAttendance',
                              'RegionWinner_South_AvgScore', 'RegionWinner_South_Enrollment_Male', 'RegionWinner_South_Enrollment_Female',
                              'RegionWinner_Midwest', 'RegionWinner_Midwest_WinLossRatio', 'RegionWinner_Midwest_AvgAttendance',
                              'RegionWinner_Midwest_AvgScore', 'RegionWinner_Midwest_Enrollment_Male', 'RegionWinner_Midwest_Enrollment_Female',
                              # Additional features
                              'MonthsBetween']]
y_train = finals_training_df['NationalChampion']

# Define test data (from finals_test_df)
X_test = finals_test_df[['CustomerDMACode', 
                         # Semifinal winners and their stats
                         'SemifinalWinner_East_West', 'SemifinalWinner_East_West_WinLossRatio', 'SemifinalWinner_East_West_AvgAttendance',
                         'SemifinalWinner_East_West_AvgScore', 'SemifinalWinner_East_West_Enrollment_Male', 'SemifinalWinner_East_West_Enrollment_Female',
                         'SemifinalWinner_South_Midwest', 'SemifinalWinner_South_Midwest_WinLossRatio', 'SemifinalWinner_South_Midwest_AvgAttendance',
                         'SemifinalWinner_South_Midwest_AvgScore', 'SemifinalWinner_South_Midwest_Enrollment_Male', 'SemifinalWinner_South_Midwest_Enrollment_Female',
                         # Region winners and their stats
                         'RegionWinner_East', 'RegionWinner_East_WinLossRatio', 'RegionWinner_East_AvgAttendance',
                         'RegionWinner_East_AvgScore', 'RegionWinner_East_Enrollment_Male', 'RegionWinner_East_Enrollment_Female',
                         'RegionWinner_West', 'RegionWinner_West_WinLossRatio', 'RegionWinner_West_AvgAttendance',
                         'RegionWinner_West_AvgScore', 'RegionWinner_West_Enrollment_Male', 'RegionWinner_West_Enrollment_Female',
                         'RegionWinner_South', 'RegionWinner_South_WinLossRatio', 'RegionWinner_South_AvgAttendance',
                         'RegionWinner_South_AvgScore', 'RegionWinner_South_Enrollment_Male', 'RegionWinner_South_Enrollment_Female',
                         'RegionWinner_Midwest', 'RegionWinner_Midwest_WinLossRatio', 'RegionWinner_Midwest_AvgAttendance',
                         'RegionWinner_Midwest_AvgScore', 'RegionWinner_Midwest_Enrollment_Male', 'RegionWinner_Midwest_Enrollment_Female',
                         # Additional features
                         'MonthsBetween']]

# Define categorical features
cat_features = ['CustomerDMACode', 'SemifinalWinner_East_West', 'SemifinalWinner_South_Midwest',
                'RegionWinner_East', 'RegionWinner_West', 'RegionWinner_South', 'RegionWinner_Midwest']

# Set up the parameter grid for grid search
param_grid = {
    'iterations': [300, 400],
    'depth': [10, 11],
    'learning_rate': [0.1, 0.05],
    'l2_leaf_reg': [3, 5]
}

# Initialize CatBoost classifier
model = CatBoostClassifier(
    cat_features=cat_features,
    verbose=False,
    random_seed=42
)

# Set up the grid search
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=3,
    scoring='accuracy',
    verbose=2,
    n_jobs=-1
)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best model
finals_best_model = grid_search.best_estimator_

print(f"Best Parameters: {grid_search.best_params_}")

Fitting 3 folds for each of 16 candidates, totalling 48 fits


KeyboardInterrupt: 

In [45]:
# Predict the National Champion using the best model
finals_test_df['NationalChampion'] = finals_best_model.predict(X_test).flatten()
finals_test_df.head()


,CustomerDMACode,RegionWinner_East,RegionWinner_East_Wins,RegionWinner_East_Losses,RegionWinner_East_WinLossRatio,RegionWinner_East_AvgAttendance,RegionWinner_East_AvgScore,RegionWinner_East_Enrollment_Male,RegionWinner_East_Enrollment_Female,RegionWinner_West,RegionWinner_West_Wins,RegionWinner_West_Losses,RegionWinner_West_WinLossRatio,RegionWinner_West_AvgAttendance,RegionWinner_West_AvgScore,RegionWinner_West_Enrollment_Male,RegionWinner_West_Enrollment_Female,RegionWinner_South,RegionWinner_South_Wins,RegionWinner_South_Losses,RegionWinner_South_WinLossRatio,RegionWinner_South_AvgAttendance,RegionWinner_South_AvgScore,RegionWinner_South_Enrollment_Male,RegionWinner_South_Enrollment_Female,RegionWinner_Midwest,RegionWinner_Midwest_Wins,RegionWinner_Midwest_Losses,RegionWinner_Midwest_WinLossRatio,RegionWinner_Midwest_AvgAttendance,RegionWinner_Midwest_AvgScore,RegionWinner_Midwest_Enrollment_Male,RegionWinner_Midwest_Enrollment_Female,SemifinalWinner_East_West,SemifinalWinner_East_West_Wins,SemifinalWinner_East_West_Losses,SemifinalWinner_East_West_WinLossRatio,SemifinalWinner_East_West_AvgAttendance,SemifinalWinner_East_West_AvgScore,SemifinalWinner_East_West_Enrollment_Male,SemifinalWinner_East_West_Enrollment_Female,SemifinalWinner_South_Midwest,SemifinalWinner_South_Midwest_Wins,SemifinalWinner_South_Midwest_Losses,SemifinalWinner_South_Midwest_WinLossRatio,SemifinalWinner_South_Midwest_AvgAttendance,SemifinalWinner_South_Midwest_AvgScore,SemifinalWinner_South_Midwest_Enrollment_Male,SemifinalWinner_South_Midwest_Enrollment_Female,NCAACustomerRecordCreated,BracketEntryCreatedDate,MonthsBetween,NationalChampion
BracketEntryId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2074118,560,UConn,31,3,10.333333,14017.88,81.47,10645,11834,North Carolina,27,7,3.857143,15767.47,81.47,8068,12174,Houston,30,4,7.500000,9347.35,73.03,18290,19653,Purdue,29,4,7.250000,13329.06,83.39,21670,16279,North Carolina,27,7,3.857143,15767.47,81.47,8068,12174,Houston,30,4,7.500000,9347.35,73.03,18290,19653,2020-03-29,2024-03-19,48,North Carolina
2692634,820,UConn,31,3,10.333333,14017.88,81.47,10645,11834,North Carolina,27,7,3.857143,15767.47,81.47,8068,12174,Duke,24,8,3.000000,11475.81,79.84,3137,3448,Kansas,22,10,2.200000,13952.66,75.25,8859,9845,UConn,31,3,10.333333,14017.88,81.47,10645,11834,Duke,24,8,3.000000,11475.81,79.84,3137,3448,2024-04-02,2024-03-20,-1,UConn
1252684,529,Iowa St.,27,7,3.857143,12059.44,75.56,14070,11171,Arizona,25,8,3.125000,12118.18,87.94,13090,17292,Kentucky,23,9,2.555556,17427.94,89.44,9596,13127,Creighton,23,9,2.555556,13651.44,80.53,1789,2501,Iowa St.,27,7,3.857143,12059.44,75.56,14070,11171,Kentucky,23,9,2.555556,17427.94,89.44,9596,13127,2021-12-08,2024-03-18,27,Iowa St.
1950205,819,UConn,31,3,10.333333,14017.88,81.47,10645,11834,North Carolina,27,7,3.857143,15767.47,81.47,8068,12174,Houston,30,4,7.500000,9347.35,73.03,18290,19653,Purdue,29,4,7.250000,13329.06,83.39,21670,16279,UConn,31,3,10.333333,14017.88,81.47,10645,11834,Houston,30,4,7.500000,9347.35,73.03,18290,19653,2023-03-22,2024-03-19,12,UConn
2756293,504,UConn,31,3,10.333333,14017.88,81.47,10645,11834,North Carolina,27,7,3.857143,15767.47,81.47,8068,12174,Marquette,25,9,2.777778,14084.65,78.29,3328,4200,Creighton,23,9,2.555556,13651.44,80.53,1789,2501,UConn,31,3,10.333333,14017.88,81.47,10645,11834,Marquette,25,9,2.777778,14084.65,78.29,3328,4200,2024-02-20,2024-03-20,1,UConn


In [49]:
# Create a dictionary to map institution names to their IDs
institution_name_to_id = dict(zip(institutions['InstitutionName'], institutions['InstitutionID']))

# Replace institution names with their IDs in the finals_test_df
finals_test_df['SemifinalWinner_East_West'] = finals_test_df['SemifinalWinner_East_West'].map(institution_name_to_id)
finals_test_df['SemifinalWinner_South_Midwest'] = finals_test_df['SemifinalWinner_South_Midwest'].map(institution_name_to_id)
finals_test_df['NationalChampion'] = finals_test_df['NationalChampion'].map(institution_name_to_id)

# Select specific columns from finals_test_df
selected_columns = finals_test_df[['SemifinalWinner_East_West', 'SemifinalWinner_South_Midwest', 'NationalChampion']]

# Export the selected columns to a CSV file
selected_columns.to_csv('selected_finals_test_df.csv', index=True)



In [50]:
selected_columns = pd.read_csv('selected_finals_test_df.csv')
selected_columns.head()

,BracketEntryId,SemifinalWinner_East_West,SemifinalWinner_South_Midwest,NationalChampion
0,2074118,457,288,457
1,2692634,164,193,164
2,1252684,311,334,311
3,1950205,164,288,164
4,2756293,164,387,164
